## 一、项目背景
飞桨框架中的[PaddleDetection](https://github.com/PaddlePaddle/PaddleDetection)套件可以很好的帮助我们解决目标检测任务.

[PP-YOLOE](https://github.com/PaddlePaddle/PaddleDetection/tree/release/2.5/configs/ppyoloe)是基于PP-YOLOv2的卓越的单阶段Anchor-free模型，超越了多种流行的YOLO模型。PP-YOLOE有一系列的模型，即s/m/l/x，可以通过width multiplier和depth multiplier配置。PP-YOLOE避免了使用诸如Deformable Convolution或者Matrix NMS之类的特殊算子，以使其能轻松地部署在多种多样的硬件上，总结下来，PPYOLOE模型就是又快又好。
#### **因此，本文采用PaddleDetection套件中的PP-YOLOE模型在钢铁缺陷检测数据集中进行训练与推理**
根据PaddleDetection给出的云端模型性能对比，各模型结构和骨干网络的代表模型在COCO数据集上精度mAP和单卡Tesla V100上预测速度(FPS)对比图如下：

![https://gitee.com/paddlepaddle/PaddleDetection/raw/develop/docs/images/fps_map.png](https://gitee.com/paddlepaddle/PaddleDetection/raw/develop/docs/images/fps_map.png)



## 二、环境准备
### 2.1 数据准备
本次采用钢铁缺陷检测数据集COCO格式进行训练


In [ ]:
# 解压数据集（仅运行一次）
!tar -xvf data/data206842/NEU-DET-coco.tar -C ./data/

In [ ]:
!unzip data/data165665/dataset.zip -d ./work/

In [6]:
# 组织数据目录
!mkdir MyDataset
!mkdir MyDataset/JPEGImages
!mkdir MyDataset/annotations

In [7]:
!mv data/NEU-DET-COCO/train/* MyDataset/JPEGImages/
!mv data/NEU-DET-COCO/test/* MyDataset/JPEGImages/
!mv data/NEU-DET-COCO/val/* MyDataset/JPEGImages/
!mv data/NEU-DET-COCO/annotations/* MyDataset/annotations/

### 2.2 安装PaddleDetection以及依赖

In [ ]:
!pip install paddlex

In [ ]:
!git clone https://gitee.com/paddlepaddle/PaddleDetection.git -b release/2.5

正克隆到 'PaddleDetection'...
remote: Enumerating objects: 257246, done.
remote: Counting objects: 100% (2243/2243), done.
remote: Compressing objects: 100% (1303/1303), done.
接收对象中:  75% (193018/257246), 390.43 MiB | 56.00 KiB/s    


In [39]:
%cd ~

/home/aistudio


In [2]:
%cd PaddleDetection

/home/aistudio/PaddleDetection


In [ ]:
# 安装依赖
!pip install -r requirements.txt --user
!python setup.py install --user

## 三、模型选型
PaddleDetection提供了多种模型进行选择，具体的模型选型可以根据自己数据集的情况选取精度与速度相适应的模型，本次采用**ppyoloe_crn_l_300e_coco**


### 3.1 找到配置文件
配置文件在PaddleDetection/configs/ppyoloe/ppyoloe_crn_l_300e_coco.yml，主要包含五个部分，为了方便更改，将其copy到根目录下
![](https://ai-studio-static-online.cdn.bcebos.com/5012b253517d440e965a66b000ae1a31a3500764fb3c4cb389345c3295502e28)

### 3.2 修改配置文件
> 找到配置文件后，需要对其进行修改，参考链接https://github.com/PaddlePaddle/PaddleDetection/blob/release/2.6/docs/advanced_tutorials/customization/detection.md

在这里主要修改：
* 预训练模型改为  pretrain_weights: https://paddledet.bj.bcebos.com/models/ppyoloe_crn_l_300e_coco.pdparams  加速模型收敛
*  coco_detection 文件修改数据集路径
* 学习率训练轮数等超参数

In [4]:
# 训练配置文件覆盖
!cp ../ppyoloe_crn_l_300e_coco.yml configs/ppyoloe/ppyoloe_crn_l_300e_coco.yml
!cp ../coco_detection.yml configs/datasets/coco_detection.yml
!cp ../runtime.yml configs/runtime.yml
!cp ../optimizer_300e.yml configs/ppyoloe/_base_/optimizer_300e.yml
!cp ../ppyoloe_crn.yml configs/ppyoloe/_base_/ppyoloe_crn.yml
!cp ../ppyoloe_reader.yml configs/ppyoloe/_base_/ppyoloe_reader.yml

## 四、模型训练
训练100轮map为0.44

In [6]:
# 开始训练
# 恢复训练-r output/ppyoloe_crn_l_alpha_largesize_80e_visdrone/best_model 
!python tools/train.py -c configs/ppyoloe/ppyoloe_crn_l_300e_coco.yml --use_vdl=True --vdl_log_dir=./smalldet/visdrone/ --eval \
-r output/ppyoloe_crn_l_300e_coco/best_model.pdparams

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
W0420 15:45:46.919328 22617 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 8.0, Driver API Version: 11.2, Runtime API Version: 11.2
W0420 15:45:46.922601 22617 gpu_resources.cc:91] device: 0, cuDNN Version: 8.2.
[04/20 15:45:50] ppdet.utils.checkpoint INFO: Finish resuming model weights: output/ppyoloe_crn_l_300e_coco/best_model.pdparams
[04/20 15:45:54] ppdet.engine INFO: Epoch: [85] [  0/157] learning_rate: 0.000075 loss: 1.601687 loss_cls: 0.848021 loss_iou: 0.115048 loss_dfl: 0.932090 loss_l1: 0.390656 eta: 2:10:25 batch_cost: 3.3231 data_cost: 0.0003 ips: 2.4074 images/s
[04/20 15:46:13] ppdet.engine INFO: Epoch: [85] [ 50/157] learning_rate: 0.000072 loss: 1.775180 loss_cls: 0.918659 loss_iou: 0.154577 loss_dfl: 0.987777 loss_l1: 0.496141 eta: 0:14:57 batch_cost: 0.3306 data_cost: 0.0012 ips: 24.1993 images/s
[04/20 15:46:33] ppdet.engine INFO: Epoch: [85] [100/157] learning_r

### 4.1 训练20轮模型评估
![](https://ai-studio-static-online.cdn.bcebos.com/5bb02f02618d4ac4bd0020b51708bb941b88f36d794948d8ac13c836c9948079)


### 4.2 可视化
![](https://ai-studio-static-online.cdn.bcebos.com/cadd60186efe4491b4c9bd5c4ff951141d4ef6c726fd4bb6b3bc9bbfff170408)
![](https://ai-studio-static-online.cdn.bcebos.com/63d943f5926a4c1e932f86c6b4d203b2576a78f21f414c8b9acf53d851f757e2)
![](https://ai-studio-static-online.cdn.bcebos.com/6b8efa64f6c345a0852aed92ac40bf00bb7bc069fd4140568a854b4e2b9eef51)


## 五、预测推理
![](https://ai-studio-static-online.cdn.bcebos.com/0a0ee6d18fdc48bca3b9dfc240da33b934122b34d1de472396bf56754cb9a234)
![](https://ai-studio-static-online.cdn.bcebos.com/6597c912ab5448ff895d288940484d4a8fb6b5a66fee4605bae9ce5e13137b61)
![](https://ai-studio-static-online.cdn.bcebos.com/afeec94322b34dbc8c791778472ec369a1b21e884c794ce5a7c1f8ced0095b06)


In [ ]:
# 挑一张验证集的图片展示预测效果
!python tools/infer.py \
-c configs/ppyoloe/ppyoloe_crn_l_300e_coco.yml \
-o weights=output/ppyoloe_crn_l_300e_coco/best_model \
--infer_img=../MyDataset/JPEGImages/rolled-in_scale_139.jpg \
--save_results=True

In [ ]:
# 批量预测
!python tools/infer.py \
-c configs/ppyoloe/ppyoloe_crn_l_300e_coco.yml \
-o weights=output/ppyoloe_crn_l_300e_coco/best_model \
--infer_dir=../work/dataset/test/IMAGES/ \
--output_dir=../work/dataset/test/result/ \
--draw_threshold=0.5 \
--save_results=True

## 六、部署模型

### 6.1导出部署模型

In [3]:
!python tools/export_model.py \
-c configs/ppyoloe/ppyoloe_crn_l_300e_coco.yml \
-o weights=output/ppyoloe_crn_l_300e_coco/best_model

[04/29 21:14:59] ppdet.utils.checkpoint INFO: Finish loading model weights: output/ppyoloe_crn_l_300e_coco/best_model.pdparams
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
[04/29 21:15:00] ppdet.engine INFO: Export inference config file to output_inference/ppyoloe_crn_l_300e_coco/infer_cfg.yml
[04/29 21:15:22] ppdet.engine INFO: Export model and saved in output_inference/ppyoloe_crn_l_300e_coco


### 6.2fastdeploy部署

In [ ]:
# 安装相关包
!pip install numpy opencv-python fastdeploy-gpu-python -f https://www.paddlepaddle.org.cn/whl/fastdeploy.html --user

In [1]:
# 验证部署是否正确
import fastdeploy as fd
import cv2
option = fd.RuntimeOption()
option.use_gpu()
model = fd.vision.detection.PPYOLOE("/home/aistudio/PaddleDetection/output_inference/ppyoloe_crn_l_300e_coco/model.pdmodel", "/home/aistudio/PaddleDetection/output_inference/ppyoloe_crn_l_300e_coco/model.pdiparams", "/home/aistudio/PaddleDetection/output_inference/ppyoloe_crn_l_300e_coco/infer_cfg.yml", runtime_option=option)

image = cv2.imread("/home/aistudio/1400.jpg")
result = model.predict(image)
vis_im = fd.vision.vis_detection(image, result, labels=["crazing","inclusion","patches","pitted_surface","rolled-in_scale","scratches"],score_threshold=0.4)
cv2.imwrite("/home/aistudio/work/result.png", vis_im)

[INFO] fastdeploy/vision/common/processors/transform.cc(45)::FuseNormalizeCast	Normalize and Cast are fused to Normalize in preprocessing pipeline.
[INFO] fastdeploy/vision/common/processors/transform.cc(93)::FuseNormalizeHWC2CHW	Normalize and HWC2CHW are fused to NormalizeAndPermute  in preprocessing pipeline.
[INFO] fastdeploy/vision/common/processors/transform.cc(159)::FuseNormalizeColorConvert	BGR2RGB and NormalizeAndPermute are fused to NormalizeAndPermute with swap_rb=1
[INFO] fastdeploy/runtime/runtime.cc(293)::CreateOrtBackend	Runtime initialized with Backend::ORT in Device::GPU.


True

### 6.3运行launch/demo.gradio.py
![](https://ai-studio-static-online.cdn.bcebos.com/067295323257490e8199761ac7538f38c9a376b6973d4cb99ed7fbcc8707934c)
